In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.init as init
import random

datasets.MNIST('~/Documents/ipynb/datasets',train=True,download=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                       ]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: /home/kami/Documents/ipynb/datasets
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [5]:
import subprocess

result = subprocess.run(['build/dcgan'], capture_output=True, text=True)

output = result.stdout

lines = output.split('\n')
parsing = False
cpp_array_elapsed_time = []
cpp_total=[0,0,0]
for line in lines:
    if line.startswith('========'):
        parsing = True
    elif parsing:
        if line.startswith('STEP'):
            _, _, time = line.split(':')
            cpp_array_elapsed_time.append(int(time))
        elif line.startswith('Total Time'):
            cpp_total[0] = int(line.split(':')[1])
        elif line.startswith('Sum Time'):
            cpp_total[1] = int(line.split(':')[1])
        elif line.startswith('Step Count'):
            cpp_total[2] = int(line.split(':')[1])

print(output)
print(cpp_array_elapsed_time)
print(cpp_total)

첫 번째 배치의 데이터 텐서 초기값:
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.2460
-0.4115
 1.5359
 2.7960
 0.7213
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
-0.4242
[ CPUFloatType{28} ]

첫 번째 배치의 타겟 텐서:
5
[ CPULongType{} ]
0
[ CPULongType{} ]
4
[ CPULongType{} ]
1
[ CPULongType{} ]
9
[ CPULongType{} ]
2
[ CPULongType{} ]
parameter:0.07008072734
[ CPUFloatType{} ]
Epoch: 0 | Batch: 100 | Loss: 0.2333426178
Epoch: 0 | Batch: 200 | Loss: 0.3861721754
Epoch: 0 | Batch: 300 | Loss: 0.2468242049
Epoch: 0 | Batch: 400 | Loss: 0.3281974196
Epoch: 0 | Batch: 500 | Loss: 0.2015509754
Epoch: 0 | Batch: 600 | Loss: 0.2389870733
Epoch: 0 | Batch: 700 | Loss: 0.131513074
Epoch: 0 | Batch: 800 | Loss: 0.1957902908
Epoch: 0 | Batch: 900 | Loss: 0.07153533399
Epoch: 1 | Batch: 100 | Loss: 0.04624519497
Epoch: 1 | Batch: 200 | Loss: 0.1075298786
Epoch: 1 | Batch: 300 | Loss: 0.09305275232
Epoch: 1 | Batch: 400 | Loss: 0.2868

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.init as init
import random
import time

torch.set_printoptions(precision=10)

# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        # self.init_weights()
    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(self.fc3(x), dim=1)

    def init_weights(self):
        nn.init.constant_(self.fc1.weight, 0.5)
        nn.init.constant_(self.fc1.bias, 0.1)
        nn.init.constant_(self.fc2.weight, 0.5)
        nn.init.constant_(self.fc2.bias, 0.1)
        nn.init.constant_(self.fc3.weight, 0.5)
        nn.init.constant_(self.fc3.bias, 0.1)



def set_random():
    torch.manual_seed(1)
    torch.cuda.manual_seed_all(1)
    random.seed(1)


def measure_and_execute(task,  array_elapsed_time, time_step):
    start_time = time.time()
    result = task()
    end_time = time.time()
    elapsed_time = end_time - start_time
    array_elapsed_time[time_step] += elapsed_time
    return result, time_step + 1

def next_batch(train_loader_iter):
    try:
        return next(train_loader_iter)
    except StopIteration:
        return None



def main():


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using {device} device')


    set_random()
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data',train=True,download=False,
                       transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64,shuffle=False,num_workers=4,pin_memory=True)
    for batch_idx, (data, target) in enumerate(train_loader):
     print("첫 번째 배치의 데이터 텐서 초기값:")
     print(data[0,0,10])  # 첫 번째 데이터 텐서

     print("\n첫 번째 배치의 타겟 텐서:")
     print(target[0])
     print(target[1])
     print(target[2])
     print(target[3])
     print(target[4])
     print(target[5])
     break


    set_random()
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    print("VALUE:" + str(sum(list(model.parameters())[0].data[0])))
    print(list(model.parameters())[0].data.dtype)


    TOTAL_STEP=8
    global array_elapsed_time
    array_elapsed_time = [0 for i in range(TOTAL_STEP)]
    model.train()
    total_epoch_time_start = time.time()
    global total_count
    total_count = 0
    for epoch in range(0, 10):
        train_loader_iter = iter(train_loader)
        batch_idx=0
        while True:
            TIME_STEP = 0

            batch = measure_and_execute(
                lambda: next(train_loader_iter, None),
                array_elapsed_time,
                TIME_STEP
            )
            if batch[0] is None:
                break

            total_count=total_count+1
            batch_idx=batch_idx+1
            (data, target), TIME_STEP = batch

            (data,target), TIME_STEP = measure_and_execute(
              lambda: (data.to(device), target.to(device)),
              array_elapsed_time,
              TIME_STEP
            )


            _, TIME_STEP = measure_and_execute(
                optimizer.zero_grad,
                array_elapsed_time,
                TIME_STEP
            )

            output, TIME_STEP = measure_and_execute(
                lambda: model(data),
                array_elapsed_time,
                TIME_STEP
            )

            loss, TIME_STEP = measure_and_execute(
                lambda: F.nll_loss(output, target),
                array_elapsed_time,
                TIME_STEP
            )

            _, TIME_STEP = measure_and_execute(
                lambda: loss.backward(),
                array_elapsed_time,
                TIME_STEP
            )

            _, TIME_STEP = measure_and_execute(
                optimizer.step,
                array_elapsed_time,
                TIME_STEP
            )

            if batch_idx % 100 == 0 and batch_idx != 0:
              _, TIME_STEP = measure_and_execute(
                lambda: print(f'Epoch: {epoch} | Batch: {batch_idx} | Loss: {loss.item():.6f}'),
                array_elapsed_time,
                TIME_STEP
              )


    total_epoch_time_end = time.time()
    global total_epoch_elapsed_time
    total_epoch_elapsed_time= total_epoch_time_end - total_epoch_time_start
    print("==================================")
    print(f"Total Time: {total_epoch_elapsed_time}")
    for i in range(TOTAL_STEP):
      print(f"STEP {i}: array_elapsed_time:{array_elapsed_time[i]}")
    print(f"Sum Time: {sum(array_elapsed_time)}")
    print(f"Step Count:{total_count}")

if __name__ == '__main__':
    main()



Using cpu device
첫 번째 배치의 데이터 텐서 초기값:
tensor([-0.4242129624, -0.4242129624, -0.4242129624, -0.4242129624,
        -0.4242129624, -0.4242129624, -0.4242129624, -0.4242129624,
        -0.4242129624, -0.2460176945, -0.4114847183,  1.5359351635,
         2.7960302830,  0.7213280797, -0.4242129624, -0.4242129624,
        -0.4242129624, -0.4242129624, -0.4242129624, -0.4242129624,
        -0.4242129624, -0.4242129624, -0.4242129624, -0.4242129624,
        -0.4242129624, -0.4242129624, -0.4242129624, -0.4242129624])

첫 번째 배치의 타겟 텐서:
tensor(5)
tensor(0)
tensor(4)
tensor(1)
tensor(9)
tensor(2)
VALUE:tensor(0.0700804144)
torch.float32
Epoch: 0 | Batch: 100 | Loss: 0.233343
Epoch: 0 | Batch: 200 | Loss: 0.386172
Epoch: 0 | Batch: 300 | Loss: 0.246825
Epoch: 0 | Batch: 400 | Loss: 0.328197
Epoch: 0 | Batch: 500 | Loss: 0.201205
Epoch: 0 | Batch: 600 | Loss: 0.240446
Epoch: 0 | Batch: 700 | Loss: 0.129086
Epoch: 0 | Batch: 800 | Loss: 0.207704
Epoch: 0 | Batch: 900 | Loss: 0.071838
Epoch: 1 | Batch

In [8]:
total_epoch_elapsed_time

128.15136551856995

In [11]:
data = {
    "Func":
    [
    "Total Time",
    "Summed Time",
    "Step Count",
    "Data Loading",
    "Copying To GPU",
    "Optimizer Init",
    "Model Forward",
    "Loss Calculation",
    "Backward Pass",
    "Parameter Update",
    "Progress Printing"
    ],

    "Python(s)":
    [
        total_epoch_elapsed_time,
        sum(array_elapsed_time),
        total_count,

    ] + array_elapsed_time,
    "CPP(s)":
     [
        cpp_total[0],
        cpp_total[1],
        cpp_total[2],


    ] + cpp_array_elapsed_time,

}





In [12]:
import pandas as pd
pd.set_option('display.float_format', '{:.4f}'.format)
df = pd.DataFrame(data)
df["CPP(s)"] = df.apply(lambda row: row["CPP(s)"] if row["Func"] == "Step Count" else row["CPP(s)"] / 1000000, axis=1)
df['diff(s)'] = df['Python(s)'] - df['CPP(s)']
df["Time Difference (%)"] = ((df["Python(s)"] - df["CPP(s)"]) / df["CPP(s)"]) * 100

df




ImportError: cannot import name 'broadcast_to' from 'numpy.lib.stride_tricks' (/home/kami/anaconda3/envs/env/lib/python3.12/site-packages/numpy/lib/stride_tricks.py)